# Exam project of group 26
### Group members:
- Baltazar Dydensborg
- Johan Kielgast Ladelund
- Laura Weile
- Simon Juul Hansen

### Research Question:
???

### All of the dependency imports

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime
from tqdm import tqdm
import warnings


## Scraping betting data from Oddsportal.com

In [ ]:
# Example of URLs
newest_season_url = 'https://www.oddsportal.com/soccer/denmark/superliga/results/'
previous_season_url = lambda year_span: f'https://www.oddsportal.com/soccer/denmark/superliga-{year_span}/results/'
previous_season_url('2020-2021')
prev_url_example = 'https://www.oddsportal.com/soccer/denmark/superliga-2020-2021/results/'

In [ ]:
# requesting html data
page = requests.get('https://www.oddsportal.com/soccer/denmark/superliga-2020-2021/results/#/page/2/')
soup = BeautifulSoup(page.content, 'html.parser')
soup.find('body')

## FIRST PROBLEM WITH ODDSPORTAL.COM:
### The HTML data is being retrieved via a JavaScript Query, so we can't directly get it via scraping the website with 'requests'.
### We need to find a solution to getting the HTML data via the JS query, possible via Selenium

# Elpriser fra Nordpool

In [241]:
url = 'https://www.nordpoolgroup.com/api/downloads/4675'
el_res = requests.get(url)
el_json = el_res.json()
el_json.keys()

dict_keys(['Files', 'AllCategories', 'AllTypes', 'AllResolutions', 'DateWithTime', 'ShowCategoryColumn'])

In [258]:
hourly = [i for i in el_json['Files'] if i['Resolution'] == 'Hourly']
elspot = [i for i in hourly if i['Categories'] == ['Elspot Prices']]
elspot = [i for i in elspot if i['Name'].endswith('DKK')]
elspot_url = ['https://www.nordpoolgroup.com'+i['Url'] for i in elspot]
elspot_url

['https://www.nordpoolgroup.com/4abf83/globalassets/marketdata-excel-files/elspot-prices_2013_hourly_dkk.xls',
 'https://www.nordpoolgroup.com/4abf6a/globalassets/marketdata-excel-files/elspot-prices_2014_hourly_dkk.xls',
 'https://www.nordpoolgroup.com/490b38/globalassets/marketdata-excel-files/elspot-prices_2015_hourly_dkk.xls',
 'https://www.nordpoolgroup.com/48ddf7/globalassets/marketdata-excel-files/elspot-prices_2016_hourly_dkk.xls',
 'https://www.nordpoolgroup.com/48cd09/globalassets/marketdata-excel-files/elspot-prices_2017_hourly_dkk.xls',
 'https://www.nordpoolgroup.com/48cd7b/globalassets/marketdata-excel-files/elspot-prices_2018_hourly_dkk.xls',
 'https://www.nordpoolgroup.com/48c975/globalassets/marketdata-excel-files/elspot-prices_2019_hourly_dkk.xls',
 'https://www.nordpoolgroup.com/48c954/globalassets/marketdata-excel-files/elspot-prices_2020_hourly_dkk.xls',
 'https://www.nordpoolgroup.com/4a2f7d/globalassets/marketdata-excel-files/elspot-prices_2021_hourly_dkk.xls']

In [280]:
df = pd.read_html(elspot_url[0])[0].droplevel(level = 0, axis = 1).droplevel(level = 0, axis = 1)
df[df.columns[2:]] = df[df.columns[2:]]/100

In [293]:
d = {}
for i in range(len(elspot_url)):
    d[f'DF_{2013+i}'] = pd.read_html(elspot_url[i])[0].droplevel(level = 0, axis = 1).droplevel(level = 0, axis = 1)

In [335]:
df_concat = pd.concat(d)

In [341]:
df_new = df_concat[['Unnamed: 0_level_2', 'Hours', 'SYS','DK1','DK2']].droplevel(level = 0).reset_index(drop = True)
df_new[df_new.columns[2:]] = df_new[df_new.columns[2:]]/100
df_new = df_new.rename(columns = {'Unnamed: 0_level_2': 'date'})
df_new['date'] = pd.to_datetime(df_new['date'])
df_new['Hours'] = df_new['Hours'].str[5:]
df_new['Hours'] = pd.to_datetime(df_new['Hours'], format = '%H').dt.hour

In [342]:
df_new

,date,Hours,SYS,DK1,DK2
0,2013-01-01,1,231.62,104.66,104.66
1,2013-01-01,2,227.29,82.50,82.50
2,2013-01-01,3,215.73,63.41,63.41
3,2013-01-01,4,207.97,0.75,0.75
4,2013-01-01,5,201.11,14.99,14.99
...,...,...,...,...,...
75507,2021-12-08,20,564.37,1078.45,1078.45
75508,2021-12-08,21,554.32,1071.01,1071.01
75509,2021-12-08,22,552.02,991.95,991.95
75510,2021-12-08,23,544.73,885.07,885.07


# Vejr data fra DMI API

In [ ]:
# Constants

metObsAPIKey = 'c4503ba1-28d4-45c5-850a-974e98bbb3e0'
climateDataAPIKEy = 'ac27b332-bde2-4138-a53e-f0ca82cf3667'
stationId = 6183

dmi_url= lambda API_KEY: f'https://dmigw.govcloud.dk/v2/metObs/collections/observation/items?limit=300000&stationId=06184&parameterId=temp_dry&bbox=7,54,16,58&datetime=2015-01-01T00:00:00Z/2021-08-01T00:00:00Z&api-key={API_KEY}'
dmi_url_id= lambda API_KEY, id: f'https://dmigw.govcloud.dk/v2/metObs/collections/observation/items?limit=300000&stationId=06184&parameterId={id}&bbox=7,54,16,58&datetime=2015-01-01T00:00:00Z/2021-08-01T00:00:00Z&api-key={API_KEY}'
dmi_url_allID= lambda API_KEY: f'https://dmigw.govcloud.dk/v2/metObs/collections/observation/items?limit=300000&stationId=06184&bbox=7,54,16,58&datetime=2015-01-01T00:00:00Z/2021-08-01T00:00:00Z&api-key={API_KEY}'


In [ ]:
def metObsData(stat, start_date, end_date, stationId = '06183', apiKey = metObsAPIKey):
    """
    Function that takes a parameter ID and searches for all observations from DMI
    for the given dates, where the date format is given as 'YYYY-MM-DD'
    """

    # Define constants to be used in the function
    url= lambda API_KEY, id, startDate, endDate, stationId: f'https://dmigw.govcloud.dk/v2/metObs/collections/observation/items?limit=300000&stationId={stationId}&parameterId={id}&bbox=7,54,16,58&datetime={startDate}T00:00:00Z/{endDate}T00:00:00Z&api-key={API_KEY}'
    local_counter = 1
    ErrorCounter = 0
    stationCounter = 0
    stationList = ['06184', '06186', '06187', '06188']

    while True:
        try:
            temp_url = url(apiKey, stat, start_date, end_date, stationId)
            ErrorCounter += 1

            json_dmi = requests.get(temp_url).json()
            prop = [item['properties'] for item in json_dmi['features']]
            temp_df = pd.DataFrame(prop)
            
            temp_df['observed'].drop_duplicates(inplace = True)
            temp_df.index = pd.to_datetime(temp_df['observed'])
            temp_df = temp_df.tz_convert('CET')

            if temp_df.index[-1].date() != datetime.date.fromisoformat(start_date):
                            
                if local_counter == 1:
                    df = temp_df.copy()
                    local_counter += 1
                elif local_counter > 1:
                    df = pd.concat([df,temp_df])
                else:
                    warnings.warn("Something went wrong with the local counter in IF")

                end_date = temp_df.index[-1].date().strftime("%Y-%m-%d")
                continue
            else:
                if local_counter == 1:
                    df = temp_df
                elif local_counter > 1:
                    df = pd.concat([df,temp_df])
                else:
                    warnings.warn("Something went wrong with the local counter in ELSE")

                break
            


        except KeyError as e:

            try:
                stationId = stationList[stationCounter]
                stationCounter += 1
                continue
            except IndexError as i:
                if ErrorCounter == 1:
                    warnings.warn(f"The requested stat {stat} isn't available for any default stations")
                else:
                    warnings.warn(f"The requested stat {stat} isn't available for any default stations for the daterange: {start_date} - {end_date}")
                return None
        
    return df

        
    

In [ ]:
def climateData(stat, start_date, end_date, stationId = '06184',apiKey = climateDataAPIKEy):
    """
    Function that takes a parameter ID and searches for all observations from DMI
    for the given dates, where the date format is given as 'YYYY-MM-DD'
    """

    # Define constants to be used in the function
    url= lambda API_KEY, id, startDate, endDate, stationId: f'https://dmigw.govcloud.dk/v2/climateData/collections/stationValue/items?timeResolution=hour&limit=300000&stationId={stationId}&parameterId={id}&datetime={startDate}T00:00:00Z/{endDate}T00:00:00Z&api-key={API_KEY}'
    local_counter = 1
    ErrorCounter = 0
    stationCounter = 0
    stationList = ['06181', '06186', '06187', '06188']

    while True:
        try:
            temp_url = url(apiKey, stat, start_date, end_date, stationId)
            ErrorCounter += 1

            json_dmi = requests.get(temp_url).json()
            prop = [item['properties'] for item in json_dmi['features']]
            temp_df = pd.DataFrame(prop)
            
            temp_df['to'].drop_duplicates(inplace = True)
            temp_df.index = pd.to_datetime(temp_df['to'])
            #temp_df = temp_df.tz_convert('CET')

            if temp_df.index[-1].date() != datetime.date.fromisoformat(start_date):
                            
                if local_counter == 1:
                    df = temp_df.copy()
                    local_counter += 1
                elif local_counter > 1:
                    df = pd.concat([df,temp_df])
                else:
                    warnings.warn("Something went wrong with the local counter in IF")

                end_date = temp_df.index[-1].date().strftime("%Y-%m-%d")
                warnings.warn('Loop loop date wrong')
                continue
            else:
                if local_counter == 1:
                    df = temp_df
                elif local_counter > 1:
                    df = pd.concat([df,temp_df])
                else:
                    warnings.warn("Something went wrong with the local counter in ELSE")

                break
            


        except KeyError as e:

            try:
                stationId = stationList[stationCounter]
                stationCounter += 1
                warnings.warn('Loop loop station wrong')
                continue
            except IndexError as i:
                if ErrorCounter == 1:
                    warnings.warn(f"The requested stat {stat} isn't available for any default stations")
                else:
                    warnings.warn(f"The requested stat {stat} isn't available for any default stations for the daterange: {start_date} - {end_date}")
                return None
        
    return df


In [ ]:
def TransformMetObsData(df):
    df = df
    df['hour'] = df.index.hour
    df['date'] = df.index.date
    df = df.drop_duplicates(['parameterId', 'date','hour'])\
            .drop(['created', 'stationId'], axis = 1)\
            .sort_values(by = ['date', 'hour'], ascending = [False, False])\
            .copy()
    
    df_new = df.groupby(['parameterId', 'date', 'hour'])['value'].mean()\
            .unstack(level = 0)\
            .reset_index().rename(columns={df.index.name:None})

    return df_new
    

In [ ]:
def transformClimateData(df):
    df = df
    df = df.tz_convert('CET')
    df['hour'] = df.index.hour
    df['date'] = df.index.date
    df = df.drop_duplicates(['parameterId', 'date','hour'])\
            .drop(['calculatedAt', 'created', 'from', 'qcStatus', 'timeResolution', 'validity'], axis = 1)\
            .sort_values(by = ['date', 'hour'], ascending = [False, False])\
            .copy()
    
    df_new = df.groupby(['parameterId', 'date', 'hour'])['value'].mean()\
            .unstack(level = 0)\
            .reset_index().rename(columns={df.index.name:None})

    return df_new

In [ ]:
list_of_stats=['wind_min', 'wind_max', 'visibility', 'cloud_cover', 'cloud_height', 'temp_dry', 'wind_speed', 'wind_dir', 'pressure', 'humidity']

dict_df = dict()
for stat in tqdm(list_of_stats):
    dict_df[stat] = metObsData(stat, '2015-01-01', '2021-08-01')

new_df_met = pd.concat(dict_df.values())
transform_df_met = TransformMetObsData(new_df_met)


In [ ]:
list_of_stats_climate =['bright_sunshine', 'mean_radiation', 'mean_pressure', 'mean_wind_dir', 'acc_precip', 'temp_grass', 'mean_relative_hum', 'mean_temp']

dict_df_climate = dict()
for stat in tqdm(list_of_stats_climate):
    dict_df_climate[stat] = climateData(stat, '2015-01-01', '2021-08-01')

new_df_climate = pd.concat(dict_df_climate.values())
transform_df_climate = transformClimateData(new_df_climate)

In [ ]:
def metObsPipeline(stats, start_date, end_date):
    dict_df = dict()
    for stat in tqdm(stats):
        dict_df[stat] = metObsData(stat, start_date, end_date)

    new_df_met = pd.concat(dict_df.values())
    transform_df_met = TransformMetObsData(new_df_met)
    return transform_df_met

In [ ]:
def climatePipeline(stats, start_date, end_date):
    dict_df = dict()
    for stat in tqdm(stats):
        dict_df[stat] = climateData(stat, start_date, end_date)

    new_df_climate = pd.concat(dict_df.values())
    transform_df_climate = transformClimateData(new_df_climate)
    return transform_df_climate

In [ ]:
def merger(df_met, df_climate):
    merge_df = df_met.merge(df_climate, on = ['date', 'hour'], how = 'outer')
    merge_df = merge_df.sort_values(by = ['date', 'hour']).reset_index(drop = True)
    return merge_df

In [ ]:
def total_pipeline(met_stats, climate_stats, start_date, end_date):
    df_met = metObsPipeline(met_stats, start_date, end_date)
    df_climate = climatePipeline(climate_stats, start_date, end_date)
    df_total = merger(df_met, df_climate)
    return df_total

In [345]:
for i in merge_df.columns:
    num_na = len(merge_df[i]) - len(merge_df[i].dropna())
    print(f"{i}: {num_na}")

date: 0
hour: 0
cloud_cover: 41739
cloud_height: 28619
humidity: 3535
pressure: 3034
temp_dry: 3254
visibility: 4382
wind_dir: 3035
wind_max: 3045
wind_speed: 3035
acc_precip: 304
bright_sunshine: 2
mean_pressure: 352
mean_radiation: 2
mean_relative_hum: 885
mean_temp: 329
mean_wind_dir: 13285
temp_grass: 260


In [352]:
merge_df[pd.isna(merge_df['mean_temp'])].head(100)[50:]

parameterId,date,hour,cloud_cover,cloud_height,humidity,pressure,temp_dry,visibility,wind_dir,wind_max,wind_speed,acc_precip,bright_sunshine,mean_pressure,mean_radiation,mean_relative_hum,mean_temp,mean_wind_dir,temp_grass
20986,2019-03-10,14,NaN,450.0,82.0,994.8,4.1,50000.0,329.0,9.0,7.2,NaN,0.5,NaN,147.0,NaN,NaN,NaN,3.5
20987,2019-03-10,13,NaN,NaN,70.0,994.8,5.4,30000.0,290.0,9.5,7.5,NaN,43.0,NaN,498.0,NaN,NaN,NaN,4.7
20988,2019-03-10,12,NaN,800.0,70.0,995.2,5.3,45000.0,283.0,9.3,7.8,NaN,25.5,NaN,364.0,NaN,NaN,NaN,5.0
20989,2019-03-10,11,NaN,800.0,73.0,995.4,5.3,40000.0,280.0,10.6,9.2,NaN,11.5,NaN,249.0,NaN,NaN,NaN,4.2
20990,2019-03-10,10,NaN,NaN,79.0,995.9,5.0,35000.0,277.0,10.6,8.8,NaN,40.5,NaN,322.0,NaN,NaN,NaN,3.1
20991,2019-03-10,9,NaN,NaN,79.0,996.0,4.8,23000.0,283.0,12.9,11.0,NaN,0.0,NaN,57.0,NaN,NaN,NaN,4.1
20992,2019-03-10,8,NaN,NaN,85.0,996.0,4.4,21000.0,297.0,12.5,10.2,NaN,22.5,NaN,70.0,NaN,NaN,NaN,2.4
20993,2019-03-10,7,NaN,NaN,82.0,995.6,4.3,21000.0,292.0,18.2,14.3,NaN,0.0,NaN,2.0,NaN,NaN,NaN,1.5
20994,2019-03-10,6,NaN,NaN,77.0,995.8,4.4,17000.0,287.0,13.4,10.9,NaN,0.0,NaN,0.0,NaN,NaN,NaN,2.2
20995,2019-03-10,5,NaN,NaN,76.0,994.8,4.7,17000.0,305.0,18.7,14.3,NaN,0.0,NaN,0.0,NaN,NaN,NaN,3.0


# CVR REGNSKABER 

In [ ]:
cvr_url = 'https://datacvr.virk.dk/data/offentliggorelse?dl_ref=amNsb3VkczovLzAzLzZhL2I5L2FlLzQyLzMyZjAtNDFmOS1iOWQ0LTY1ZGQ0OTc5ZGJkNg'
cvr_res = requests.get(cvr_url)
soup = BeautifulSoup(cvr_res.content, 'lxml')

In [213]:
soup.text

"Årsrapport88216512BESTSELLER A/SFredskovvej 57330 Brande2019-08-012020-07-312018-08-012019-07-312020-10-0788216512BESTSELLER A/SFredskovvej57330BrandeDKDanmark1982-08-30Ikast-BrandeEY30700228Morten FriisState Authorised Public AccountantVærkmestergade258000Aarhus CDKDanmark2020-10-07Thomas Børglum JensenRegnskabsklasse C, stor virksomhedRevisionspåtegningThe Supervisory Board and Executive Board have today discussed and approved the annual report of BESTSELLER A/S for the financial year 1 August 2019 - 31 July 2020.The annual report is prepared in accordance with the Danish Financial Statements Act.In our opinion, the consolidated financial statements and Parent financial statements give a true and fair view of the company and the Group financial position at 31 July 2020 and of the results of the Group and the Parent company operations and cash flows for the financial year 1 August 2019 - 31 July 2020.In our opinion, Management's review includes a fair review of the matters dealt with

In [212]:
lem_url = 'https://datacvr.virk.dk/data/offentliggorelse?dl_ref=amNsb3VkczovLzAzL2E3L2IyLzcxLzRiLzc4YmMtNDUwOS05OGUxLTA5ODQ2ZDVjOWZlYw'
lem_res = requests.get(lem_url)
lem_soup = BeautifulSoup(lem_res.content, 'lxml')
lem_soup.text

'iso4217:DKKxbrli:pure266643652020-01-012020-12-31266643652020-01-012020-12-311266643652020-01-012020-12-312266643652020-01-012020-12-313266643652020-01-012020-12-311266643652019-01-012019-12-31266643652020-01-012020-12-31fsa:RetainedEarningsMember266643652019-01-012019-12-31fsa:RetainedEarningsMember266643652020-12-31266643652019-12-31ÅrsrapportIngen bistandfalseRegnskabsklasse BtruefalsefalseLem Tømrer-og Snedkerforretning A/SRingvej 2 - 4, Lem7860Spøttrup26664365Skive182020-01-012020-12-312019-01-012019-12-312021-05-10Randi HansenRandi HansenFinn HansenKnud HansenFinn HansenRevisionsanpartsselskabet NesselwangOverlundvej  468830 Tjele    27988393Sparekassen VestSallingVesterbrogade 24 - Lem7860SpøttrupAdvokatfirmaet DahlGravene8800ViborgBestyrelse og direktion har dags dato behandlet og vedtaget årsrapporten for 2020 inkl. ledelsesberetningen for selskabet Lem Tømrer-og Snedkerforretning A/S.<br/><br/>Den interne årsrapport er aflagt i overensstemmelse med årsregnskabsloven og suppl